In [18]:
import pandas as pd
celiac_df=pd.read_csv("/Users/admin/Downloads/celiac_disease_lab_data.csv")
import seaborn as sns
sns.set_theme(style="whitegrid")
import matplotlib.pyplot as plt

In [19]:
celiac_df.head()

,Age,Gender,Diabetes,Diabetes Type,Diarrhoea,Abdominal,Short_Stature,Sticky_Stool,Weight_loss,IgA,IgG,IgM,Marsh,cd_type,Disease_Diagnose
0,10,Male,Yes,Type 1,inflammatory,yes,PSS,no,no,1.30,10.0,1.00,marsh type 0,potential,yes
1,9,Male,Yes,Type 1,fatty,yes,PSS,no,no,1.50,12.5,1.30,marsh type 3a,atypical,yes
2,8,Female,Yes,Type 1,watery,yes,Variant,yes,yes,0.40,8.0,0.50,marsh type 1,latent,yes
3,10,Male,Yes,Type 1,watery,yes,PSS,no,no,0.98,9.0,0.66,marsh type 3a,silent,yes
4,9,Male,Yes,Type 1,fatty,yes,PSS,no,no,1.00,10.5,1.10,marsh type 1,latent,yes


In [20]:
celiac_df.describe()

,Age,IgA,IgG,IgM
count,2206.000000,2206.000000,2206.000000,2206.000000
mean,12.768812,1.427384,10.052901,1.236963
std,7.434250,1.110042,2.047683,0.447216
min,1.000000,0.340000,5.000000,0.500000
25%,8.000000,1.000000,8.700000,0.980000
50%,10.000000,1.100000,10.000000,1.100000
75%,15.000000,1.800000,12.000000,1.500000
max,35.000000,9.000000,15.300000,2.700000


In [21]:
# Filter the DataFrame for "yes" diagnosis
yes_diagnosis_df = celiac_df[celiac_df['Disease_Diagnose'] == 'yes']

# Generate descriptive statistics for "yes" diagnosis
yes_diagnosis_stats = yes_diagnosis_df.describe()

# Filter the DataFrame for "no" diagnosis
no_diagnosis_df = celiac_df[celiac_df['Disease_Diagnose'] == 'no']

# Generate descriptive statistics for "no" diagnosis
no_diagnosis_stats = no_diagnosis_df.describe()

# Print the statistics
print("Statistics for 'yes' diagnosis:")
print(yes_diagnosis_stats)

print("\nStatistics for 'no' diagnosis:")
print(no_diagnosis_stats)

Statistics for 'yes' diagnosis:
               Age          IgA          IgG          IgM
count  1843.000000  1843.000000  1843.000000  1843.000000
mean     13.234943     1.305708    10.157027     1.231812
std       7.856934     0.546986     1.982584     0.467183
min       1.000000     0.340000     5.000000     0.500000
25%       8.000000     1.000000     8.700000     0.910000
50%      10.000000     1.100000    10.000000     1.100000
75%      16.000000     1.800000    12.000000     1.500000
max      35.000000     2.600000    15.300000     2.700000

Statistics for 'no' diagnosis:
              Age         IgA         IgG         IgM
count  363.000000  363.000000  363.000000  363.000000
mean    10.402204    2.045152    9.524242    1.263113
std      3.976199    2.350532    2.281263    0.326851
min      3.000000    0.340000    5.000000    0.600000
25%      8.000000    1.000000    8.000000    1.000000
50%      9.000000    1.300000    9.400000    1.100000
75%     12.000000    1.900000   12.0

In [22]:
# Drop the specified columns
columns_to_drop = ['Weight_loss', 'IgM', 'IgG', 'Gender', 'Age', 'Marsh', 'cd_type']
filtered_df = celiac_df.drop(columns_to_drop, axis=1)

# Display the resulting DataFrame
print(filtered_df.head())

  Diabetes Diabetes Type     Diarrhoea Abdominal Short_Stature Sticky_Stool  \
0      Yes        Type 1  inflammatory       yes           PSS           no   
1      Yes        Type 1        fatty        yes           PSS           no   
2      Yes        Type 1        watery       yes      Variant           yes   
3      Yes        Type 1        watery       yes           PSS           no   
4      Yes        Type 1        fatty        yes           PSS           no   

    IgA Disease_Diagnose  
0  1.30              yes  
1  1.50              yes  
2  0.40              yes  
3  0.98              yes  
4  1.00              yes  


In [23]:
filtered_df.describe()

,IgA
count,2206.000000
mean,1.427384
std,1.110042
min,0.340000
25%,1.000000
50%,1.100000
75%,1.800000
max,9.000000


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Separate the target variable from the features
X = filtered_df.drop('Disease_Diagnose', axis=1)
y = filtered_df['Disease_Diagnose']

# One-hot encode the categorical features
X_encoded = pd.get_dummies(X)

# Encode the target variable ('Disease_Diagnose') using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# Train a Decision Tree model using Bagging for aggregation
base_model = DecisionTreeClassifier(random_state=42)
bagging_model = BaggingClassifier(base_model, n_estimators=10, random_state=42)
bagging_model.fit(X_train, y_train)

# Evaluate the model's performance on the testing set
y_pred = bagging_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9773755656108597


In [29]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

# Assuming y_true contains the true labels and y_pred contains the predicted labels
# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred) # y_test will contain the true labels (actual values) of the target variable, and y_pred will contain the predicted labels (predicted values) of the target variable

# Display the confusion matrix
print("Confusion Matrix:")
print(cm)

#print(label_encoder.get_params())

# Calculate F1 score
f1 = f1_score(y_test, y_pred, labels=[0, 1], average='binary')
print("F1 Score:", f1)

Confusion Matrix:
[[ 60   9]
 [  1 372]]
{}
F1 Score: 0.986737400530504


In [30]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

# Summarize class distribution before undersampling
print("Class distribution before undersampling:", Counter(filtered_df['Disease_Diagnose']))

# Define the undersample strategy
undersample = RandomUnderSampler(sampling_strategy=0.5, random_state=42)

# Separate features and target variable
X = filtered_df.drop('Disease_Diagnose', axis=1)  # Features
y = filtered_df['Disease_Diagnose']  # Target variable

# Perform random undersampling
X_under, y_under = undersample.fit_resample(X, y)

# Combine the undersampled features and target variable into a new DataFrame
filtered_undersampled1_df = pd.concat([pd.DataFrame(X_under, columns=X.columns), pd.Series(y_under, name='Disease_Diagnose')], axis=1)

# Summarize class distribution after undersampling
print("Class distribution after undersampling:", Counter(filtered_undersampled1_df['Disease_Diagnose']))

Class distribution before undersampling: Counter({'yes': 1843, 'no': 363})
Class distribution after undersampling: Counter({'yes': 726, 'no': 363})


In [31]:
filtered_undersampled1_df.describe()

,IgA
count,1089.000000
mean,1.562700
std,1.469439
min,0.340000
25%,1.000000
50%,1.200000
75%,1.900000
max,9.000000


In [32]:
# Separate the target variable from the features
X = filtered_undersampled1_df.drop('Disease_Diagnose', axis=1)
y = filtered_undersampled1_df['Disease_Diagnose']

# One-hot encode the categorical features
X_encoded = pd.get_dummies(X)

# Encode the target variable ('Disease_Diagnose') using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# Train a Decision Tree model using Bagging for aggregation
base_model = DecisionTreeClassifier(random_state=42)
bagging_model = BaggingClassifier(base_model, n_estimators=10, random_state=42)
bagging_model.fit(X_train, y_train)

# Evaluate the model's performance on the testing set
y_pred = bagging_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9036697247706422


In [33]:
# y_true contains the true labels and y_pred contains the predicted labels
# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred) # y_test will contain the true labels (actual values) of the target variable, and y_pred will contain the predicted labels (predicted values) of the target variable

# Display the confusion matrix
print("Confusion Matrix:")
print(cm)

f1 = f1_score(y_test, y_pred, labels=[0, 1], average='binary')
print("F1 Score:", f1)

Confusion Matrix:
[[ 63  13]
 [  8 134]]
F1 Score: 0.9273356401384083


In [34]:
# Define the undersample strategy
undersample = RandomUnderSampler(sampling_strategy=0.6, random_state=42)

# Separate features and target variable
X = filtered_df.drop('Disease_Diagnose', axis=1)  # Features
y = filtered_df['Disease_Diagnose']  # Target variable

# Perform random undersampling
X_under, y_under = undersample.fit_resample(X, y)

# Combine the undersampled features and target variable into a new DataFrame
filtered_undersampled2_df = pd.concat([pd.DataFrame(X_under, columns=X.columns), pd.Series(y_under, name='Disease_Diagnose')], axis=1)

# Summarize class distribution after undersampling
print("Class distribution after undersampling:", Counter(filtered_undersampled2_df['Disease_Diagnose']))

Class distribution after undersampling: Counter({'yes': 605, 'no': 363})


In [35]:
# Separate the target variable from the features
X = filtered_undersampled2_df.drop('Disease_Diagnose', axis=1)
y = filtered_undersampled2_df['Disease_Diagnose']

# One-hot encode the categorical features
X_encoded = pd.get_dummies(X)

# Encode the target variable ('Disease_Diagnose') using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train-test split, of the 605:363 it could split differently compared to previous splits so how many it's testing/training on for each model will be different- search for train/test split that is the same 
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# Train a Decision Tree model using Bagging for aggregation
base_model = DecisionTreeClassifier(random_state=42)
bagging_model = BaggingClassifier(base_model, n_estimators=10, random_state=42)
bagging_model.fit(X_train, y_train)

# Evaluate the model's performance on the testing set
y_pred = bagging_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9123711340206185


In [36]:
# Assume y_true contains the true labels and y_pred contains the predicted labels
# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred) # y_test will contain the true labels (actual values) of the target variable, and y_pred will contain the predicted labels (predicted values) of the target variable

# Display the confusion matrix
print("Confusion Matrix:")
print(cm)

f1 = f1_score(y_test, y_pred, labels=[0, 1], average='binary')
print("F1 Score:", f1)

Confusion Matrix:
[[ 60  15]
 [  2 117]]
F1 Score: 0.9322709163346613


In [37]:
# Define the undersample strategy
undersample = RandomUnderSampler(sampling_strategy=0.4, random_state=42)

# Separate features and target variable
X = filtered_df.drop('Disease_Diagnose', axis=1)  # Features
y = filtered_df['Disease_Diagnose']  # Target variable

# Perform random undersampling
X_under, y_under = undersample.fit_resample(X, y)

# Combine the undersampled features and target variable into a new DataFrame
filtered_undersampled3_df = pd.concat([pd.DataFrame(X_under, columns=X.columns), pd.Series(y_under, name='Disease_Diagnose')], axis=1)

# Summarize class distribution after undersampling
print("Class distribution after undersampling:", Counter(filtered_undersampled3_df['Disease_Diagnose']))

Class distribution after undersampling: Counter({'yes': 907, 'no': 363})


In [38]:
# Separate the target variable from the features
X = filtered_undersampled3_df.drop('Disease_Diagnose', axis=1)
y = filtered_undersampled3_df['Disease_Diagnose']

# One-hot encode the categorical features
X_encoded = pd.get_dummies(X)

# Encode the target variable ('Disease_Diagnose') using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# Train a Decision Tree model using Bagging for aggregation
base_model = DecisionTreeClassifier(random_state=42)
bagging_model = BaggingClassifier(base_model, n_estimators=10, random_state=42)
bagging_model.fit(X_train, y_train)

# Evaluate the model's performance on the testing set
y_pred = bagging_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.937007874015748


In [39]:
# y_true contains the true labels and y_pred contains the predicted labels
# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred) # y_test will contain the true labels (actual values) of the target variable, and y_pred will contain the predicted labels (predicted values) of the target variable

# Display the confusion matrix
print("Confusion Matrix:")
print(cm)

f1 = f1_score(y_test, y_pred, labels=[0, 1], average='binary')
print("F1 Score:", f1)

Confusion Matrix:
[[ 67  12]
 [  4 171]]
F1 Score: 0.9553072625698324


In [62]:
y_encoded[0:1]

array([0])

In [63]:
y[0:1]

11    no
Name: Disease_Diagnose, dtype: object